In [8]:
import tensorflow as tf
import os
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
import nengo
import nengo_dl

In [9]:
# training params
batch_size = 128
epochs = 3
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

# load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
class_count = 10

# reshape for spiking nn
x_train = x_train.reshape((x_train.shape[0], 1, -1))
y_train = y_train.reshape((y_train.shape[0], 1, -1))
x_test = x_test.reshape((x_test.shape[0], 1, -1))
y_test = y_test.reshape((y_test.shape[0], 1, -1))


In [11]:
input_layer = Input(shape=(28, 28, 1))
conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')(input_layer)
conv2 = Conv2D(filters=64, kernel_size=3, strides=2, activation='relu')(conv1)
flatten = Flatten()(conv2)
dense = Dense(128, activation='relu')(flatten)
output_layer = Dense(class_count)(dense)

tensorflow_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

converter = nengo_dl.Converter(tensorflow_model)
simulator = nengo_dl.Simulator(converter.net, minibatch_size=batch_size)

simulator.compile(optimizer=tf.optimizers.RMSprop(.001),
                      loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[tf.metrics.sparse_categorical_accuracy])

simulator.fit(
        { converter.inputs[input_layer]: x_train },
        { converter.outputs[output_layer]: y_train },
        validation_data=(
            { converter.inputs[input_layer]: x_test },
            { converter.outputs[output_layer]: y_test }
        ),
        epochs=epochs

    )

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/3
468/468 [==============================] - 9s 19ms/step - loss: 0.4074 - probe_loss: 0.4074 - probe_sparse_categorical_accuracy: 0.9483

C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\simulator.py:2109: RuntimeWarning: Simulator with model=Model: <Network "functional_3">, dt=0.001000 was deallocated while open. Simulators should be closed manually to ensure resources are properly freed.
  RuntimeWarning,
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\simulator.py:1932: UserWarning: Number of elements in input data (60000) is not evenly divisible by Simulator.minibatch_size (128); input data will be truncated.
  % (data_batch, self.minibatch_size)


TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
params_folder = os.path.join('..', 'snn-params')
os.makedirs(params_folder,exist_ok=True)
params_file = os.path.join(params_folder, 'keras_snn_params')

simulator.save_params(params_file)
print('params of ANN saved')


In [ ]:
activation = nengo.RectifiedLinear()
steps = 10
scale_firing_rates = 1
synapse = None
number_test = 400

nengo_converter = nengo_dl.Converter(tensorflow_model, swap_activations={ tf.nn.relu: activation })


nengo_input = nengo_converter.inputs[input_layer]
nengo_output = nengo_converter.outputs[output_layer]

sample_neurons_conv1 = np.linspace(0, np.prod(conv1.shape[1:]), 1000, endpoint=False, dtype=np.int32)
sample_neurons_conv2 = np.linspace(0, np.prod(conv2.shape[1:]), 1000, endpoint=False, dtype=np.int32)


with nengo_converter.net:
    conv1_probe = nengo.Probe(nengo_converter.layers[conv1][sample_neurons_conv1])
    conv2_probe = nengo.Probe(nengo_converter.layers[conv2][sample_neurons_conv2])

tiled_x_test = np.tile(x_test[:number_test], (1, steps, 1))

with nengo_converter.net:
    nengo_dl.configure_settings(stateful=False)

with nengo_dl.Simulator(nengo_converter.net, minibatch_size=10, progress_bar=False) as nengo_simulator:
    nengo_simulator.load_params(params_file)
    data = nengo_simulator.predict({ nengo_input: tiled_x_test })